# Output Parsers

In [1]:
import pprint as pp
from pathlib import Path
from dotenv import find_dotenv, load_dotenv

from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

load_dotenv(Path('../../.env'))

True

Creating an object of `CommaSeparatedListOutputParser`

In [2]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [4]:
# Injecting the expected format into the
# prompt template text
prompt = PromptTemplate(
    template="Provide 5 examples of {query}.\n{format_instructions}",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)

In [10]:
llm = OpenAI(temperature=.9, model="text-davinci-003")
prompt = prompt.format(query="Currencies")

output = llm(prompt)

print(prompt)
print()
print(output)

Provide 5 examples of Currencies.
Your response should be a list of comma separated values, eg: `foo, bar, baz`



US Dollar, Euro, Japanese Yen, British Pound, Indian Rupee


### Json Format

In [11]:
response_schemas = [
    ResponseSchema(name="currency", description="answer to the user's question"),
    ResponseSchema(name="abbrevation", description="Whats the abbrebation of that currency")
]

In [12]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


print(format_instructions)
print(output_parser)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // answer to the user's question
	"abbrevation": string  // Whats the abbrebation of that currency
}
```
response_schemas=[ResponseSchema(name='currency', description="answer to the user's question", type='string'), ResponseSchema(name='abbrevation', description='Whats the abbrebation of that currency', type='string')]


In [14]:
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{query}",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions}
)

In [15]:
print(prompt)

input_variables=['query'] output_parser=None partial_variables={'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"currency": string  // answer to the user\'s question\n\t"abbrevation": string  // Whats the abbrebation of that currency\n}\n```'} template='answer the users question as best as possible.\n{format_instructions}\n{query}' template_format='f-string' validate_template=True


In [16]:
prompt = prompt.format(query="what's the currency of America?")

In [17]:
print(prompt)

answer the users question as best as possible.
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"currency": string  // answer to the user's question
	"abbrevation": string  // Whats the abbrebation of that currency
}
```
what's the currency of America?


In [18]:
output = llm(prompt)
print(output)



```json
{
	"currency": "US Dollar"
	"abbrevation": "USD"
}
```
